In [21]:
import cPickle
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit

import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import operator
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

from pandas_datareader import data
import pandas_datareader as pdr
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
import re
from dateutil import parser
from bt import Strategy#, Portfolio
import quandl

In [22]:
def getStock(symbol, start, end):
    """
    Downloads Stock from Yahoo Finance.
    Computes daily Returns based on Adj Close.
    Returns pandas dataframe.
    """
    
    df =  pdr.data.get_data_yahoo(symbol, start, end)

    df.columns.values[-1] = 'AdjClose'
    df.columns = df.columns + '_' + symbol
    df['Return_%s' %symbol] = df['AdjClose_%s' %symbol].pct_change()
    
    print(df)
    
    return df

d1 = datetime.datetime(2000,5,1)
d2 = datetime.datetime(2017,5,1)

df = getStock("AAPL", d1, d2)

df.columns

             Open_AAPL   High_AAPL    Low_AAPL  Close_AAPL  Volume_AAPL  \
Date                                                                      
2000-05-01  124.874994  125.125000  121.875006  124.312498     56548800   
2000-05-02  123.250004  126.250006  117.500006  117.874994     59108000   
2000-05-03  118.937502  121.249998  111.624998  115.062500    122449600   
2000-05-04  115.124998  115.249994  110.562497  110.687500     99878800   
2000-05-05  110.812503  114.749996  110.718702  113.125006     71019200   
2000-05-08  112.093700  113.687502  110.000001  110.124997     46225200   
2000-05-09  110.312498  111.250003  104.874997  105.437500     81785200   
2000-05-10  104.062502  105.000000   98.749997   99.312500    133772800   
2000-05-11  101.374997  104.250003   99.000003  102.812500    124936000   
2000-05-12  106.000003  110.499999  104.765602  107.625000     76728400   
2000-05-15  108.062500  108.062500  100.125002  101.000002    169733200   
2000-05-16  104.515603  1

Index([u'Open_AAPL', u'High_AAPL', u'Low_AAPL', u'Close_AAPL', u'Volume_AAPL',
       u'AdjClose_AAPL', u'Return_AAPL'],
      dtype='object')

In [23]:
def getStockDataFromWeb(fout, start_string, end_string):
    """
    Collects predictors data from Yahoo Finance and Quandl.
    Returns a list of dataframes.
    """
    start = start_string
    end = end_string
    
    sp500 = getStock('^GSPC', start, end)
    nasdaq = getStock('^IXIC', start, end)
    frankfurt = getStock('^GDAXI', start, end)
    london = getStock('^FTSE', start, end)
    paris = getStock('^FCHI', start, end)
    hkong = getStock('^HSI', start, end)
    nikkei = getStock('^N225', start, end)
    australia = getStock('^AXJO', start, end)
    djia = getStock('^DJI', start, end)
    
    out =  pdr.data.get_data_yahoo(fout, start, end)
    out.columns.values[-1] = 'AdjClose'
    out.columns = out.columns + '_Out'
    out['Return_Out'] = out['AdjClose_Out'].pct_change()
    
    return [out, nasdaq, djia, frankfurt, london, paris, hkong, nikkei, australia]

d1 = datetime.datetime(2000,4,15)
d2 = datetime.datetime(2017,5,1)

bigDF = getStockDataFromWeb('AAPL',d1, d2)

             Open_^GSPC   High_^GSPC    Low_^GSPC  Close_^GSPC  Volume_^GSPC  \
Date                                                                           
2000-04-17  1356.560059  1401.530029  1346.500000  1401.439941    1204700000   
2000-04-18  1401.439941  1441.609985  1397.810059  1441.609985    1109400000   
2000-04-19  1441.609985  1447.689941  1424.260010  1427.469971    1001400000   
2000-04-20  1427.469971  1435.489990  1422.079956  1434.540039     896200000   
2000-04-24  1434.540039  1434.540039  1407.130005  1429.859985     868700000   
2000-04-25  1429.859985  1477.670044  1429.859985  1477.439941    1071100000   
2000-04-26  1477.439941  1482.939941  1456.979980  1460.989990     999600000   
2000-04-27  1460.989990  1469.209961  1434.810059  1464.920044    1111000000   
2000-04-28  1464.920044  1473.619995  1448.150024  1452.430054     984600000   
2000-05-01  1452.430054  1481.510010  1452.430054  1468.250000     966300000   
2000-05-02  1468.250000  1468.250000  14

In [4]:
def addFeatures(dataframe, adjclose, returns, n):
    """
    operates on two columns of dataframe:
    - n >= 2
    - given Return_* computes the return of day i respect to day i-n. 
    - given AdjClose_* computes its moving average on n days

    """
    
    return_n = adjclose[9:] + "Time" + str(n)
    dataframe[return_n] = dataframe[adjclose].pct_change(n)
    
    roll_n = returns[7:] + "RolMean" + str(n)
    dataframe[roll_n] = pd.DataFrame(dataframe[returns]).rolling(window = n).mean()



In [5]:
def applyRollMeanDelayedReturns(datasets, delta):
    """
    applies rolling mean and delayed returns to each dataframe in the list
    """
    for dataset in datasets:
        columns = dataset.columns    
        adjclose = columns[-2]
        returns = columns[-1]
        for n in delta:
            addFeatures(dataset, adjclose, returns, n)
    
    return datasets    

delta = {2}

h = applyRollMeanDelayedReturns(bigDF, delta)

In [6]:
h[0]

,Open_Out,High_Out,Low_Out,Close_Out,Volume_Out,AdjClose_Out,Return_Out,OutTime2,OutRolMean2
Date,,,,,,,,,
2000-04-17,109.500003,123.937496,109.062503,123.874998,102390400,4.012297,NaN,NaN,NaN
2000-04-18,123.499996,126.875000,119.375002,126.875000,97731200,4.109467,0.024218,NaN,NaN
2000-04-19,126.187494,130.250004,119.750004,121.125002,130037600,3.923225,-0.045320,-0.022200,-0.010551
2000-04-20,123.687504,124.749998,117.062506,118.875004,180530000,3.850348,-0.018576,-0.063054,-0.031948
2000-04-24,115.000002,120.499994,114.749996,120.499994,110905200,3.902981,0.013670,-0.005160,-0.002453
2000-04-25,122.124998,128.749996,122.062500,128.312496,97910400,4.156027,0.064834,0.079390,0.039252
2000-04-26,126.624994,128.000006,119.999996,121.312496,91728000,3.929298,-0.054554,0.006743,0.005140
2000-04-27,117.187502,126.999996,116.578102,126.750004,81650800,4.105418,0.044822,-0.012177,-0.004866
2000-04-28,127.125006,127.499994,121.312496,124.062506,62395200,4.018371,-0.021203,0.022669,0.011810


In [7]:
def mergeDataframes(datasets, index):
    """
    merges datasets in the list 
    """
    subset = []#tion
    subset = [dataset.iloc[:, index:] for dataset in datasets[1:]]
    
    first = subset[0].join(subset[1:], how = 'outer')
    finance = datasets[0].iloc[:, index:].join(first, how = 'left') 
#    finance = finance[finance.index > cut]
    return finance

index = 6
"""
The cut is not useful
"""
#cut = datetime.datetime(2005,11,4)

mergedDF = mergeDataframes(bigDF, index)

mergedDF

,Return_Out,OutTime2,OutRolMean2,Return_^IXIC,^IXICTime2,^IXICRolMean2,Return_^DJI,^DJITime2,^DJIRolMean2,Return_^GDAXI,...,^FCHIRolMean2,Return_^HSI,^HSITime2,^HSIRolMean2,Return_^N225,^N225Time2,^N225RolMean2,Return_^AXJO,^AXJOTime2,^AXJORolMean2
Date,,,,,,,,,,,,,,,,,,,,,
2000-04-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04-18,0.024218,NaN,NaN,0.071884,NaN,NaN,0.017473,NaN,NaN,0.001301,...,NaN,0.034950,NaN,NaN,-0.002058,NaN,NaN,0.022555,NaN,NaN
2000-04-19,-0.045320,-0.022200,-0.010551,-0.022976,0.047257,0.024454,-0.008587,0.008736,0.004443,0.002810,...,0.008716,0.009745,0.045035,0.022347,0.006173,0.004102,0.002057,0.019566,0.042563,0.021061
2000-04-20,-0.018576,-0.063054,-0.031948,-0.016871,-0.039459,-0.019923,0.015840,0.007117,0.003626,-0.008142,...,0.007065,-0.003893,0.005813,0.002926,-0.006670,-0.000538,-0.000248,NaN,NaN,NaN
2000-04-24,0.013670,-0.005160,-0.002453,-0.044293,-0.060417,-0.030582,0.005722,0.021653,0.010781,NaN,...,NaN,NaN,NaN,NaN,0.012462,-0.025274,-0.012405,NaN,NaN,NaN
2000-04-25,0.064834,0.079390,0.039252,0.065686,0.018483,0.010696,0.020055,0.025892,0.012888,0.017122,...,0.012397,0.000838,-0.003059,-0.001528,-0.011246,0.001077,0.000608,NaN,NaN,NaN
2000-04-26,-0.054554,0.006743,0.005140,-0.021863,0.042386,0.021911,-0.016119,0.003613,0.001968,0.014840,...,0.012463,-0.009923,-0.009094,-0.004543,-0.007553,-0.018714,-0.009400,0.015672,0.035545,0.017619
2000-04-27,0.044822,-0.012177,-0.004866,0.039652,0.016922,0.008894,-0.005244,-0.021279,-0.010682,-0.022577,...,-0.005605,-0.002267,-0.012168,-0.006095,-0.006349,-0.013855,-0.006951,-0.003529,0.012088,0.006072
2000-04-28,-0.021203,0.022669,0.011810,0.022954,0.063516,0.031303,-0.014161,-0.019331,-0.009703,0.026717,...,0.002565,0.021486,0.019170,0.009609,-0.002523,-0.008857,-0.004436,0.003058,-0.000481,-0.000235


In [8]:
def applyTimeLag(dataset, lags, delta):
    """
    apply time lag to return columns selected according  to delta.
    Days to lag are contained in the lags list passed as argument.
    Returns a NaN free dataset obtained cutting the lagged dataset
    at head and tail
    """
    
    dataset.Return_Out = dataset.Return_Out.shift(-1)
    
    maxLag = max(lags)
 
    allColumns = dataset.columns[:]
  
    for column in allColumns:
        data_col = pd.Series(dataset[column])
        data_col = data_col.interpolate(method='linear', axis=0, limit=None, inplace=False, limit_direction='forward', downcast=None)
        dataset[column] = data_col

    return dataset.iloc[maxLag:-1,:]

lags = {0, 1, 2}

hhh = applyTimeLag(mergedDF, lags, delta)

hhh

,Return_Out,OutTime2,OutRolMean2,Return_^IXIC,^IXICTime2,^IXICRolMean2,Return_^DJI,^DJITime2,^DJIRolMean2,Return_^GDAXI,...,^FCHIRolMean2,Return_^HSI,^HSITime2,^HSIRolMean2,Return_^N225,^N225Time2,^N225RolMean2,Return_^AXJO,^AXJOTime2,^AXJORolMean2
Date,,,,,,,,,,,,,,,,,,,,,
2000-04-19,-0.018576,-0.022200,-0.010551,-0.022976,0.047257,0.024454,-0.008587,0.008736,0.004443,0.002810,...,0.008716,0.009745,0.045035,0.022347,0.006173,0.004102,0.002057,0.019566,0.042563,0.021061
2000-04-20,0.013670,-0.063054,-0.031948,-0.016871,-0.039459,-0.019923,0.015840,0.007117,0.003626,-0.008142,...,0.007065,-0.003893,0.005813,0.002926,-0.006670,-0.000538,-0.000248,0.018593,0.040808,0.020200
2000-04-24,0.064834,-0.005160,-0.002453,-0.044293,-0.060417,-0.030582,0.005722,0.021653,0.010781,0.004490,...,0.009731,-0.001528,0.001377,0.000699,0.012462,-0.025274,-0.012405,0.017619,0.039054,0.019340
2000-04-25,-0.054554,0.079390,0.039252,0.065686,0.018483,0.010696,0.020055,0.025892,0.012888,0.017122,...,0.012397,0.000838,-0.003059,-0.001528,-0.011246,0.001077,0.000608,0.016645,0.037299,0.018479
2000-04-26,0.044822,0.006743,0.005140,-0.021863,0.042386,0.021911,-0.016119,0.003613,0.001968,0.014840,...,0.012463,-0.009923,-0.009094,-0.004543,-0.007553,-0.018714,-0.009400,0.015672,0.035545,0.017619
2000-04-27,-0.021203,-0.012177,-0.004866,0.039652,0.016922,0.008894,-0.005244,-0.021279,-0.010682,-0.022577,...,-0.005605,-0.002267,-0.012168,-0.006095,-0.006349,-0.013855,-0.006951,-0.003529,0.012088,0.006072
2000-04-28,0.002015,0.022669,0.011810,0.022954,0.063516,0.031303,-0.014161,-0.019331,-0.009703,0.026717,...,0.002565,0.021486,0.019170,0.009609,-0.002523,-0.008857,-0.004436,0.003058,-0.000481,-0.000235
2000-05-01,-0.051785,-0.019231,-0.009594,0.025234,0.048768,0.024094,0.007255,-0.007009,-0.003453,0.022883,...,0.013716,0.020359,0.030150,0.014984,0.023889,0.021306,0.010683,-0.004429,-0.001384,-0.000685
2000-05-02,-0.023860,-0.049874,-0.024885,-0.043615,-0.019481,-0.009190,-0.007460,-0.000260,-0.000103,0.019049,...,0.024867,0.019232,0.041130,0.020359,0.001971,0.025908,0.012930,0.009381,0.004910,0.002476


In [9]:
def prepareDataForCrossValidation(dataset):
    """
    generate categorical output column, set features and return whole data set
    """
    le = preprocessing.LabelEncoder()
    
    dataset['UpDown'] = dataset['Return_Out']
    dataset.UpDown[dataset.UpDown >= 0] = 'Up'
    dataset.UpDown[dataset.UpDown < 0] = 'Down'
    dataset.UpDown = le.fit(dataset.UpDown).transform(dataset.UpDown)
    """
    Here set feature from column 3 to the end, get rid of all the colomn related to S&P500
    """
    features = dataset.columns[3:-1]
    X = dataset[features]    
    y = dataset.UpDown
    return [X,y]

[X_whole, y_whole] = prepareDataForCrossValidation(hhh)
X_whole

C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\pandas\core\generic.py:4702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ljia\Anaco

,Return_^IXIC,^IXICTime2,^IXICRolMean2,Return_^DJI,^DJITime2,^DJIRolMean2,Return_^GDAXI,^GDAXITime2,^GDAXIRolMean2,Return_^FTSE,...,^FCHIRolMean2,Return_^HSI,^HSITime2,^HSIRolMean2,Return_^N225,^N225Time2,^N225RolMean2,Return_^AXJO,^AXJOTime2,^AXJORolMean2
Date,,,,,,,,,,,,,,,,,,,,,
2000-04-19,-0.022976,0.047257,0.024454,-0.008587,0.008736,0.004443,0.002810,0.004114,0.002055,0.018258,...,0.008716,0.009745,0.045035,0.022347,0.006173,0.004102,0.002057,0.019566,0.042563,0.021061
2000-04-20,-0.016871,-0.039459,-0.019923,0.015840,0.007117,0.003626,-0.008142,-0.005355,-0.002666,0.009103,...,0.007065,-0.003893,0.005813,0.002926,-0.006670,-0.000538,-0.000248,0.018593,0.040808,0.020200
2000-04-24,-0.044293,-0.060417,-0.030582,0.005722,0.021653,0.010781,0.004490,0.001743,0.000912,0.007900,...,0.009731,-0.001528,0.001377,0.000699,0.012462,-0.025274,-0.012405,0.017619,0.039054,0.019340
2000-04-25,0.065686,0.018483,0.010696,0.020055,0.025892,0.012888,0.017122,0.008841,0.004490,0.006697,...,0.012397,0.000838,-0.003059,-0.001528,-0.011246,0.001077,0.000608,0.016645,0.037299,0.018479
2000-04-26,-0.021863,0.042386,0.021911,-0.016119,0.003613,0.001968,0.014840,0.032216,0.015981,-0.004218,...,0.012463,-0.009923,-0.009094,-0.004543,-0.007553,-0.018714,-0.009400,0.015672,0.035545,0.017619
2000-04-27,0.039652,0.016922,0.008894,-0.005244,-0.021279,-0.010682,-0.022577,-0.008072,-0.003869,-0.012339,...,-0.005605,-0.002267,-0.012168,-0.006095,-0.006349,-0.013855,-0.006951,-0.003529,0.012088,0.006072
2000-04-28,0.022954,0.063516,0.031303,-0.014161,-0.019331,-0.009703,0.026717,0.003537,0.002070,0.023967,...,0.002565,0.021486,0.019170,0.009609,-0.002523,-0.008857,-0.004436,0.003058,-0.000481,-0.000235
2000-05-01,0.025234,0.048768,0.024094,0.007255,-0.007009,-0.003453,0.022883,0.024905,0.012476,0.015619,...,0.013716,0.020359,0.030150,0.014984,0.023889,0.021306,0.010683,-0.004429,-0.001384,-0.000685
2000-05-02,-0.043615,-0.019481,-0.009190,-0.007460,-0.000260,-0.000103,0.019049,0.046274,0.022883,0.007270,...,0.024867,0.019232,0.041130,0.020359,0.001971,0.025908,0.012930,0.009381,0.004910,0.002476


In [10]:
def performClassification(X_train, y_train, X_test, y_test, method):
    """
    performs classification on daily returns using several algorithms (method).
    method --> string algorithm
    parameters --> list of parameters passed to the classifier (if any)
    fout --> string with name of stock to be predicted
    savemodel --> boolean. If TRUE saves the model to pickle file
    """
   
    if method == 'RF':   
        return performRFClass(X_train, y_train, X_test, y_test)
        
    elif method == 'KNN':
        return performKNNClass(X_train, y_train, X_test, y_test)
    
    elif method == 'SVM':   
        return performSVMClass(X_train, y_train, X_test, y_test)
    
    elif method == 'ADA':
        return performAdaBoostClass(X_train, y_train, X_test, y_test)
    
    elif method == 'GTB': 
        return performGTBClass(X_train, y_train, X_test, y_test)

    elif method == 'QDA': 
        return performQDAClass(X_train, y_train, X_test, y_test)

In [11]:
def performRFClass(X_train, y_train, X_test, y_test):
    """
    Random Forest Binary Classification
    """
    clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
    clf.fit(X_train, y_train)

    feature_importance = clf.feature_importances_
    print feature_importance
    accuracy = clf.score(X_test, y_test)
    
        
    y_predict = clf.predict(X_test)
    
    print("y_predict = ", y_predict)
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print(tn, fp, fn, tp)
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    return accuracy

#performRFClass (X_train, y_train, X_test, y_test)

In [12]:
def performKNNClass(X_train, y_train, X_test, y_test):
    """
    KNN binary Classification
    """
    clf = neighbors.KNeighborsClassifier()
    clf.fit(X_train, y_train)
    
    accuracy = clf.score(X_test, y_test)
            
    y_predict = clf.predict(X_test)    
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print '(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    
    return accuracy
#performKNNClass (X_train, y_train, X_test, y_test)

In [13]:
def performSVMClass(X_train, y_train, X_test, y_test):
    """
    SVM binary Classification
    """
    clf = SVC()
    clf.fit(X_train, y_train)

    accuracy = clf.score(X_test, y_test)
    
    y_predict = clf.predict(X_test)
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print '(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    return accuracy
#performSVMClass (X_train, y_train, X_test, y_test)

In [14]:
def performAdaBoostClass(X_train, y_train, X_test, y_test):
    """
    Ada Boosting binary Classification
    """
    clf = AdaBoostClassifier()
    clf.fit(X_train, y_train)
    
    accuracy = clf.score(X_test, y_test)

    y_predict = clf.predict(X_test)
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print '(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    return accuracy
#performAdaBoostClass (X_train, y_train, X_test, y_test)

In [15]:
def performGTBClass(X_train, y_train, X_test, y_test):
    """
    Gradient Tree Boosting binary Classification
    """
    clf = GradientBoostingClassifier(n_estimators=100)
    clf.fit(X_train, y_train)
    
    y_predict = clf.predict(X_test)
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print'(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    accuracy = clf.score(X_test, y_test)
    
    return accuracy
#performGTBClass(X_train, y_train, X_test, y_test)

In [16]:
def performQDAClass(X_train, y_train, X_test, y_test):
    """
    Quadratic Discriminant Analysis binary Classification
    """
   
    clf = QDA()
    clf.fit(X_train, y_train)
    
    accuracy = clf.score(X_test, y_test)
    y_predict = clf.predict(X_test)
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print '(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    return accuracy

#performQDAClass(X_train, y_train, X_test, y_test)

In [17]:
def timeSeriesCV(dataset_X,dataset_y, n_splits, algorithm):
    print dataset_X.shape
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    counter = 0
    accuracies = np.zeros(n_splits)
    print accuracies
    for train_index, test_index in tscv.split(dataset_X):
        counter += 1
        print 'cyle: ', counter
        print 'TRAIN size: ', train_index[-1], 'TEST size: ', test_index[-1] - test_index[0] 
    
        X_train, X_test = dataset_X[:train_index[-1]], dataset_X[test_index[0]:test_index[-1]]
        y_train, y_test = dataset_y[:train_index[-1]], dataset_y[test_index[0]:test_index[-1]]
        
        print 'X_train shape: ', X_train.shape
        print 'X_test shape: ', X_test.shape
        
        index = counter - 1
        accuracies[index] = performClassification(X_train, y_train, X_test, y_test, algorithm)
        print 'Accuracy: ', accuracies[index], '\n'
        
    print 'Average accuracy: ', np.mean(accuracies)
    return np.mean(accuracies)

    
timeSeriesCV(X_whole, y_whole, 10, 'RF')

(4283, 24)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  392 TEST size:  388
X_train shape:  (392, 24)
X_test shape:  (388, 24)
[ 0.0408645   0.04027606  0.04178313  0.04796891  0.04144828  0.04246595
  0.03673594  0.03734032  0.03706108  0.03890585  0.04150471  0.04251116
  0.0443376   0.03913646  0.03940517  0.04637018  0.04040364  0.0405783
  0.04837184  0.04244231  0.04192126  0.04237312  0.04211651  0.04367771]
('y_predict = ', array([1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 

0.50953608247422677

In [18]:
timeSeriesCV(X_whole, y_whole, 10, 'SVM')

(4283, 24)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  392 TEST size:  388
X_train shape:  (392, 24)
X_test shape:  (388, 24)
y_predict =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(tn, fp, fn, tp) : ( 204 0 184 0 )
             preci

C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


y_predict =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(tn, fp, fn, tp) : ( 172 0 216 0 )
             precision    recall  f1-score   support

       Down       0.44      1.00      0.61       172
         Up       0.00      0.00      0.00       216

avg

0.52036082474226808

In [19]:
timeSeriesCV(X_whole, y_whole, 10, 'ADA')

(4283, 24)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  392 TEST size:  388
X_train shape:  (392, 24)
X_test shape:  (388, 24)
y_predict =  [1 0 0 0 0 1 1 1 0 1 0 0 1 1 1 1 0 0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 1 1 0
 1 1 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0
 0 1 1 0 0 1 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0
 0 1 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1 1 0 0 0 0 0
 0 1 1 0 0 1 0 0 0 0 1 1 0 1 0 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1
 1 1 0 1 1 1 0 0 0 0 1 0 0 1 1 1 0 1 1 0 0 0 0 1 1 1 0 1 0 0 1 0 1 0 0 1 1
 1 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 1 1
 0 1 0 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 0 1 0 1 0 0 0 1
 1 1 0 0 0 1 0 1 0 0 0 1 1 0 1 1 0 0 0 1 1 0 1 1 0 0 1 0 0 1 1 1 0 1 0 0 1
 1 1 1 1 0 0 0 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 0 1 0 0 1 1 1 0 0 1 1 1 0 1 0
 0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 1 0 1]
(tn, fp, fn, tp) : ( 113 91 101 83 )
             pre

0.50438144329896906

In [25]:
timeSeriesCV(X_whole, y_whole, 10, 'GTB')

(260, 24)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  29 TEST size:  22
X_train shape:  (29, 24)
X_test shape:  (22, 24)
y_predict =  [0 0 1 0 1 1 1 0 0 0 1 1 1 1 0 0 1 0 1 1 1 0]
(tn, fp, fn, tp) : ( 5 5 5 7 )
             precision    recall  f1-score   support

       Down       0.50      0.50      0.50        10
         Up       0.58      0.58      0.58        12

avg / total       0.55      0.55      0.55        22

Accuracy:  0.545454545455 

cyle:  2
TRAIN size:  52 TEST size:  22
X_train shape:  (52, 24)
X_test shape:  (22, 24)
y_predict =  [0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0]
(tn, fp, fn, tp) : ( 6 2 8 6 )
             precision    recall  f1-score   support

       Down       0.43      0.75      0.55         8
         Up       0.75      0.43      0.55        14

avg / total       0.63      0.55      0.55        22

Accuracy:  0.545454545455 

cyle:  3
TRAIN size:  75 TEST size:  22
X_train shape:  (75, 24)
X_test shape:  (22, 24)
y_predict =  [1

0.50909090909090904

In [26]:
timeSeriesCV(X_whole, y_whole, 10, 'QDA')

(260, 24)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  29 TEST size:  22
X_train shape:  (29, 24)
X_test shape:  (22, 24)
y_predict =  [0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0]
(tn, fp, fn, tp) : ( 7 3 11 1 )
             precision    recall  f1-score   support

       Down       0.39      0.70      0.50        10
         Up       0.25      0.08      0.12        12

avg / total       0.31      0.36      0.30        22

Accuracy:  0.363636363636 

cyle:  2
TRAIN size:  52 TEST size:  22
X_train shape:  (52, 24)
X_test shape:  (22, 24)
y_predict =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(tn, fp, fn, tp) : ( 8 0 14 0 )
             precision    recall  f1-score   support

       Down       0.36      1.00      0.53         8
         Up       0.00      0.00      0.00        14

avg / total       0.13      0.36      0.19        22

Accuracy:  0.363636363636 

cyle:  3
TRAIN size:  75 TEST size:  22
X_train shape:  (75, 24)
X_test shape:  (22, 24)
y_predict =  

C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\sklearn\discriminant_analysis.py:695: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


 0.590909090909 

cyle:  5
TRAIN size:  121 TEST size:  22
X_train shape:  (121, 24)
X_test shape:  (22, 24)
y_predict =  [1 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 1]
(tn, fp, fn, tp) : ( 7 8 5 2 )
             precision    recall  f1-score   support

       Down       0.58      0.47      0.52        15
         Up       0.20      0.29      0.24         7

avg / total       0.46      0.41      0.43        22

Accuracy:  0.409090909091 

cyle:  6
TRAIN size:  144 TEST size:  22
X_train shape:  (144, 24)
X_test shape:  (22, 24)
y_predict =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
(tn, fp, fn, tp) : ( 9 0 12 1 )
             precision    recall  f1-score   support

       Down       0.43      1.00      0.60         9
         Up       1.00      0.08      0.14        13

avg / total       0.77      0.45      0.33        22

Accuracy:  0.454545454545 

cyle:  7
TRAIN size:  167 TEST size:  22
X_train shape:  (167, 24)
X_test shape:  (22, 24)
y_predict =  [0 0 0 0 0 0 0 0 0 0 0 1 0 0 

0.43636363636363634

In [20]:
timeSeriesCV(X_whole, y_whole, 10, 'KNN')

(4283, 24)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  392 TEST size:  388
X_train shape:  (392, 24)
X_test shape:  (388, 24)
y_predict =  [0 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 1 1 0 0 1 1 1 1 1 0 0 1 1 0
 1 1 0 1 0 1 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1
 0 0 1 1 1 0 0 1 1 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 1 1 1 0 1 1 1 0 0 1 1
 1 0 0 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 0 0 0
 0 0 1 1 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 0 0 0 1
 0 1 0 0 1 0 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0
 1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0
 0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0 0 1 1 0 1 1 1 0 0 1
 0 1 0 0 1 0 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1
 1 0 1 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 1
 0 1 0 1 1 1 0 0 0 0 1 0 1 0 0 1 1 1]
(tn, fp, fn, tp) : ( 99 105 90 94 )
             prec

0.52087628865979385